In [1]:
from dataset import TSVDataset

labelled_dataset = TSVDataset('./annotated_data_500/final_dataset_v2.tsv', supervised_like=True)

In [14]:
from torch.utils.data import Subset
from utils import Evaluator

fold = 5

train_set = Subset(labelled_dataset, [i for i in range((-1 + fold)%5, len(labelled_dataset), 5)] + [i for i in range((0 + fold)%5, len(labelled_dataset), 5)] + [i for i in range((1 + fold)%5, len(labelled_dataset), 5)])
valid_set = Subset(labelled_dataset, [i for i in range((2 + fold)%5, len(labelled_dataset), 5)])
test_set  = Subset(labelled_dataset, [i for i in range((3 + fold)%5, len(labelled_dataset), 5)])

evaluator = Evaluator(classifier='naive', mode='macro', detail=True)

len(train_set), len(valid_set), len(test_set)

(257, 86, 85)

In [3]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("all-mpnet-base-v2")

In [4]:
queries = ['solution or alleviation of the issue','human interest or emotion or personalization', 'conflict or disagreement or two sides', 'morality and religion', 'financial gains or costs or economic consequences'] 
query_embeddings = model.encode(queries)

In [5]:
from nltk.tokenize import sent_tokenize

def find_frame_with_best_n(text, n):
    sentences = sent_tokenize(text)
    embeddings = model.encode(sentences)

    cos_sim = util.cos_sim(query_embeddings, embeddings)

    all_sentence_combinations = []
    for i in range(len(queries)):
        for j in range(len(sentences)):
            all_sentence_combinations.append([cos_sim[i][j], i, j])

    #Sort list by the highest cosine similarity score
    all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)

    output = [0, 0, 0, 0, 0]
    for score, i, j in all_sentence_combinations[:n]:
        output[i] = 1

    return output

In [51]:
n = 1
preds = []
truths = []
for i in range(0, len(test_set)):
    preds.append(find_frame_with_best_n(test_set[i][0], n))
    truths.append(test_set[i][1])

input_dict = {"y_true": truths, "y_pred": preds}
from utils import Evaluator

evaluator = Evaluator(classifier='naive', mode='macro', detail=True)   
evaluator.eval(input_dict)

{'Precision': [0.23529411764705882, 0, 0.75, 0.6, 0.3333333333333333],
 'Recall': [0.13333333333333333,
  0,
  0.1016949152542373,
  0.2727272727272727,
  0.8181818181818182],
 'F1': [0.1702127659574468,
  0,
  0.17910447761194032,
  0.37499999999999994,
  0.4736842105263157],
 'Acc': [0.5411764705882353,
  0.8235294117647058,
  0.35294117647058826,
  0.8823529411764706,
  0.5294117647058824]}

In [6]:
def find_frame(text, threshold):
    sentences = sent_tokenize(text)
    embeddings = model.encode(sentences)

    cos_sim = util.cos_sim(query_embeddings, embeddings)

    all_sentence_combinations = []
    for i in range(len(queries)):
        for j in range(len(sentences)):
            all_sentence_combinations.append([cos_sim[i][j], i, j])

    #Sort list by the highest cosine similarity score
    all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)

    output = [0, 0, 0, 0, 0]
    for score, i, j in all_sentence_combinations:
        if score < threshold[i]:
            break
        else:
            output[queries.index(queries[i])] = 1

    return output

In [8]:
best = 0
record = 0
for i in range(0, 20):
    threshold = [i / 40] * 5
    preds = []
    truths = []
    for j in range(0, len(valid_set)):
        preds.append(find_frame(valid_set[j][0], threshold))
        truths.append(valid_set[j][1])

    input_dict = {"y_true": truths, "y_pred": preds}
    from utils import Evaluator

    evaluator = Evaluator(classifier='naive', mode='macro', detail=True)   

    res = evaluator.eval(input_dict)
    f1 = sum(res['F1']) / 5
    if f1 > best:
        best =  f1
        record = i

print(record / 40)


0.475


In [15]:
threshold = [0.15] * 5

preds = []
truths = []
for i in range(0, len(test_set)):
    preds.append(find_frame(test_set[i][0], threshold))
    truths.append(test_set[i][1])

input_dict = {"y_true": truths, "y_pred": preds}
import json

with open('./log/output_{}.json'.format(fold), 'w+', encoding='utf-8') as f:
    json.dump(input_dict, f)

from utils import Evaluator

evaluator = Evaluator(classifier='naive', mode='macro', detail=True)   

evaluator.eval(input_dict)

{'Precision': [0.3625,
  0.2,
  0.7283950617283951,
  0.21568627450980393,
  0.25882352941176473],
 'Recall': [0.9666666666666667, 0.9285714285714286, 1.0, 1.0, 1.0],
 'F1': [0.5272727272727272,
  0.32911392405063294,
  0.8428571428571429,
  0.3548387096774193,
  0.4112149532710281],
 'Acc': [0.38823529411764707,
  0.3764705882352941,
  0.7411764705882353,
  0.5294117647058824,
  0.25882352941176473]}